In [47]:
#default_exp word2vec

In [48]:
#export


from github_search import paperswithcode_tasks
import gensim.models
import pandas as pd

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
%cd ..

/home/kuba/Projects/github_search


In [3]:
paperswithcode_df, all_papers_df = paperswithcode_tasks.get_paperswithcode_dfs()
papers_with_readmes_df = pd.read_csv("output/papers_with_readmes.csv")

INFO - 13:15:07: Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 13:15:07: NumExpr defaulting to 8 threads.


In [52]:
papers_with_readmes_df.shape

(12224, 25)

In [4]:
!ls output/*readme*

output/papers_with_readmes.csv


In [16]:
#export


def get_sentences(papers_df, papers_with_readmes_df, max_length=1000):
    abstract_sentences = papers_df['abstract'].str.lower().str.split(" ").dropna()
    readme_sentences = papers_with_readmes_df['readme'].str.lower().str.split(" ").dropna()
    sentences = list(abstract_sentences) + list(readme_sentences)
    return [sent[:max_length] for sent in sentences]

In [17]:
sentences = get_sentences(all_papers_df, papers_with_readmes_df)

In [18]:

sentences = get_sentences(all_papers_df, papers_with_readmes_df)
lengths = pd.Series(map(len, sentences))#.describe()

In [49]:
#export


from gensim.models.callbacks import CallbackAny2Vec


class LossLogger(CallbackAny2Vec):
    '''Output loss at each epoch'''
    def __init__(self):
        self.epoch = 1
        self.losses = []

    def on_epoch_begin(self, model):
        print(f'Epoch: {self.epoch}', end='\t')

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        self.losses.append(loss)
        print(f'  Loss: {loss}')
        self.epoch += 1
        
        
        
class LossCallback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [50]:
embedding_dim = 200

In [51]:
#export

def make_w2v_model(sentences, embedding_dim=200):
    w2v_model = gensim.models.Word2Vec(size=embedding_dim, window=5, min_count=5, workers=24, callbacks=[LossCallback()])
    w2v_model.build_vocab(sentences, progress_per=10000)
    return w2v_model

In [1]:
#export


def train_abstract_readme_w2v(embedding_dim, epochs, upstream, product):
 
    paperswithcode_df, all_papers_df = paperswithcode_tasks.get_paperswithcode_dfs()
    papers_with_readmes_df = pd.read_csv(upstream["make_readmes"])
    
    sentences = get_sentences(all_papers_df, papers_with_readmes_df)
    lengths = pd.Series(map(len, sentences))#.describe()
   
    w2v_model = make_w2v_model(sentences)
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=epochs, report_delay=1, compute_loss=True)
    w2v_model.save(str(product))

In [34]:
w2v_model = make_w2v_model(sentences)

INFO - 13:22:06: collecting all words and their counts
INFO - 13:22:06: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 13:22:06: PROGRESS: at sentence #10000, processed 1533202 words, keeping 112056 word types
INFO - 13:22:06: PROGRESS: at sentence #20000, processed 3081944 words, keeping 184077 word types
INFO - 13:22:06: PROGRESS: at sentence #30000, processed 4592495 words, keeping 245351 word types
INFO - 13:22:06: PROGRESS: at sentence #40000, processed 6095252 words, keeping 302891 word types
INFO - 13:22:06: PROGRESS: at sentence #50000, processed 7600080 words, keeping 356535 word types
INFO - 13:22:06: PROGRESS: at sentence #60000, processed 9071210 words, keeping 408868 word types
INFO - 13:22:07: PROGRESS: at sentence #70000, processed 10566770 words, keeping 457473 word types
INFO - 13:22:07: PROGRESS: at sentence #80000, processed 12088703 words, keeping 502932 word types
INFO - 13:22:07: PROGRESS: at sentence #90000, processed 13668998 words, kee

In [35]:
len(w2v_model.wv.vocab)

144098

In [36]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1, compute_loss=True)

INFO - 13:22:22: training model with 24 workers on 144098 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO - 13:22:23: EPOCH 1 - PROGRESS: at 7.43% examples, 1786780 words/s, in_qsize 45, out_qsize 2
INFO - 13:22:24: EPOCH 1 - PROGRESS: at 15.66% examples, 1878979 words/s, in_qsize 47, out_qsize 0
INFO - 13:22:25: EPOCH 1 - PROGRESS: at 23.61% examples, 1876082 words/s, in_qsize 46, out_qsize 1
INFO - 13:22:26: EPOCH 1 - PROGRESS: at 32.00% examples, 1886753 words/s, in_qsize 47, out_qsize 0
INFO - 13:22:27: EPOCH 1 - PROGRESS: at 40.30% examples, 1911051 words/s, in_qsize 43, out_qsize 4
INFO - 13:22:28: EPOCH 1 - PROGRESS: at 48.32% examples, 1923741 words/s, in_qsize 47, out_qsize 0
INFO - 13:22:29: EPOCH 1 - PROGRESS: at 56.23% examples, 1926390 words/s, in_qsize 47, out_qsize 0
INFO - 13:22:30: EPOCH 1 - PROGRESS: at 65.16% examples, 1926468 words/s, in_qsize 48, out_qsize 0
INFO - 13:22:31: EPOCH 1 - PROGRESS: at 75.15% examples, 1918257 words/s,

Loss after epoch 0: 2057522.5


INFO - 13:22:36: EPOCH 2 - PROGRESS: at 7.73% examples, 1861076 words/s, in_qsize 46, out_qsize 1
INFO - 13:22:37: EPOCH 2 - PROGRESS: at 15.90% examples, 1903972 words/s, in_qsize 48, out_qsize 0
INFO - 13:22:38: EPOCH 2 - PROGRESS: at 23.87% examples, 1894421 words/s, in_qsize 47, out_qsize 0
INFO - 13:22:39: EPOCH 2 - PROGRESS: at 32.09% examples, 1897040 words/s, in_qsize 47, out_qsize 0
INFO - 13:22:40: EPOCH 2 - PROGRESS: at 40.24% examples, 1911448 words/s, in_qsize 43, out_qsize 4
INFO - 13:22:41: EPOCH 2 - PROGRESS: at 48.14% examples, 1908642 words/s, in_qsize 43, out_qsize 4
INFO - 13:22:42: EPOCH 2 - PROGRESS: at 56.35% examples, 1922902 words/s, in_qsize 44, out_qsize 3
INFO - 13:22:43: EPOCH 2 - PROGRESS: at 65.54% examples, 1927768 words/s, in_qsize 48, out_qsize 0
INFO - 13:22:44: EPOCH 2 - PROGRESS: at 75.88% examples, 1927803 words/s, in_qsize 47, out_qsize 0
INFO - 13:22:45: EPOCH 2 - PROGRESS: at 85.14% examples, 1925170 words/s, in_qsize 45, out_qsize 2
INFO - 13:2

Loss after epoch 1: 1819594.0


INFO - 13:22:49: EPOCH 3 - PROGRESS: at 7.73% examples, 1840435 words/s, in_qsize 46, out_qsize 1
INFO - 13:22:50: EPOCH 3 - PROGRESS: at 16.06% examples, 1916283 words/s, in_qsize 45, out_qsize 2
INFO - 13:22:51: EPOCH 3 - PROGRESS: at 24.25% examples, 1925502 words/s, in_qsize 46, out_qsize 1
INFO - 13:22:52: EPOCH 3 - PROGRESS: at 32.84% examples, 1944272 words/s, in_qsize 47, out_qsize 0
INFO - 13:22:53: EPOCH 3 - PROGRESS: at 40.81% examples, 1943535 words/s, in_qsize 47, out_qsize 0
INFO - 13:22:54: EPOCH 3 - PROGRESS: at 48.57% examples, 1939869 words/s, in_qsize 46, out_qsize 1
INFO - 13:22:55: EPOCH 3 - PROGRESS: at 56.88% examples, 1952650 words/s, in_qsize 46, out_qsize 1
INFO - 13:22:56: EPOCH 3 - PROGRESS: at 65.77% examples, 1944586 words/s, in_qsize 48, out_qsize 0
INFO - 13:22:57: EPOCH 3 - PROGRESS: at 75.99% examples, 1939641 words/s, in_qsize 47, out_qsize 0
INFO - 13:22:58: EPOCH 3 - PROGRESS: at 85.54% examples, 1942053 words/s, in_qsize 47, out_qsize 0
INFO - 13:2

...

INFO - 13:26:19: EPOCH 19 - PROGRESS: at 7.34% examples, 1767251 words/s, in_qsize 44, out_qsize 3
INFO - 13:26:20: EPOCH 19 - PROGRESS: at 15.72% examples, 1888360 words/s, in_qsize 47, out_qsize 0
INFO - 13:26:21: EPOCH 19 - PROGRESS: at 23.55% examples, 1872425 words/s, in_qsize 47, out_qsize 0
INFO - 13:26:22: EPOCH 19 - PROGRESS: at 31.87% examples, 1887386 words/s, in_qsize 45, out_qsize 2
INFO - 13:26:23: EPOCH 19 - PROGRESS: at 39.77% examples, 1891959 words/s, in_qsize 43, out_qsize 4
INFO - 13:26:24: EPOCH 19 - PROGRESS: at 47.84% examples, 1909108 words/s, in_qsize 47, out_qsize 0
INFO - 13:26:25: EPOCH 19 - PROGRESS: at 55.61% examples, 1908154 words/s, in_qsize 47, out_qsize 0
INFO - 13:26:26: EPOCH 19 - PROGRESS: at 64.33% examples, 1906627 words/s, in_qsize 47, out_qsize 0
INFO - 13:26:27: EPOCH 19 - PROGRESS: at 74.59% examples, 1909016 words/s, in_qsize 46, out_qsize 1
INFO - 13:26:28: EPOCH 19 - PROGRESS: at 83.98% examples, 1910309 words/s, in_qsize 45, out_qsize 2
I

Loss after epoch 18: 937938.0


INFO - 13:26:32: EPOCH 20 - PROGRESS: at 7.34% examples, 1777550 words/s, in_qsize 46, out_qsize 1
INFO - 13:26:33: EPOCH 20 - PROGRESS: at 15.10% examples, 1802488 words/s, in_qsize 47, out_qsize 0
INFO - 13:26:34: EPOCH 20 - PROGRESS: at 22.86% examples, 1814998 words/s, in_qsize 47, out_qsize 0
INFO - 13:26:35: EPOCH 20 - PROGRESS: at 30.68% examples, 1813076 words/s, in_qsize 45, out_qsize 2
INFO - 13:26:36: EPOCH 20 - PROGRESS: at 38.24% examples, 1808284 words/s, in_qsize 47, out_qsize 0
INFO - 13:26:37: EPOCH 20 - PROGRESS: at 45.56% examples, 1809384 words/s, in_qsize 46, out_qsize 1
INFO - 13:26:38: EPOCH 20 - PROGRESS: at 53.27% examples, 1814629 words/s, in_qsize 44, out_qsize 3
INFO - 13:26:39: EPOCH 20 - PROGRESS: at 61.00% examples, 1815879 words/s, in_qsize 47, out_qsize 0
INFO - 13:26:40: EPOCH 20 - PROGRESS: at 70.75% examples, 1819457 words/s, in_qsize 45, out_qsize 2
INFO - 13:26:41: EPOCH 20 - PROGRESS: at 79.99% examples, 1818181 words/s, in_qsize 47, out_qsize 0
I

Loss after epoch 19: 936440.0


(497189361, 681442220)

In [45]:
w2v_model.save("output/abstract_readme_w2v{}.bin".format(embedding_dim))

INFO - 13:26:48: saving Word2Vec object under output/abstract_readme_w2v200.bin, separately None
INFO - 13:26:48: storing np array 'vectors' to output/abstract_readme_w2v200.bin.wv.vectors.npy
INFO - 13:26:48: not storing attribute vectors_norm
INFO - 13:26:48: storing np array 'syn1neg' to output/abstract_readme_w2v200.bin.trainables.syn1neg.npy
INFO - 13:26:48: not storing attribute cum_table
INFO - 13:26:48: saved output/abstract_readme_w2v200.bin


In [38]:
!ls output

abstract_w2v100.bin
abstract_w2v100.bin.trainables.syn1neg.npy
abstract_w2v100.bin.wv.vectors.npy
abstract_w2v200.bin
abstract_w2v200.bin.trainables.syn1neg.npy
abstract_w2v200.bin.wv.vectors.npy
abstract_w2v300.bin
abstract_w2v300.bin.trainables.syn1neg.npy
abstract_w2v300.bin.wv.vectors.npy
abstract_w2v.bin
abstract_w2v.bin.trainables.syn1neg.npy
abstract_w2v.bin.wv.vectors.npy
call_igraph.pkl
data_w2v.bin
data_w2v.bin.trainables.syn1neg.npy
data_w2v.bin.wv.vectors.npy
dependency_records.csv
gnn_embeddings_fasttext_dim200_epochs2_dim200_layers2.bin
gnn_embeddings_fasttext_dim200_epochs2_dim200_layers2.bin.vectors.npy
gnn_embeddings_fasttext_dim200_epochs50_dim200_layers2.bin
gnn_embeddings_fasttext_dim200_epochs50_dim200_layers2.bin.vectors.npy
gnn_model_2_dim200_layers2.pth
gnn_model_50_dim200_layers2.pth
graph_infomax_embeddings_fasttext_dim200_epochs100_dim200_layers2.bin
graph_infomax_embeddings_fasttext_dim200_epochs100_dim200_layers2.bin.vectors.npy
graph_infomax_embeddings_fas

In [39]:
test_tasks = pd.read_csv('output/test_tasks.csv').iloc[:,0]
train_tasks = pd.read_csv('output/train_tasks.csv').iloc[:,0]

In [40]:
from mlutil.feature_extraction import embeddings

In [41]:
avg_embedder = embeddings.AverageWordEmbeddingsVectorizer(w2v_model)

In [42]:
test_task_embeddings = avg_embedder.transform(test_tasks)
train_task_embeddings = avg_embedder.transform(train_tasks)

In [43]:
task_similarities = pd.DataFrame(test_task_embeddings @ train_task_embeddings.T, columns=train_tasks, index=test_tasks)

In [44]:
task_similarities.idxmax().to_dict()

{'image classification': 'audio classification',
 'representation learning': 'sparse learning',
 'object detection': 'object recognition',
 'neural architecture search': 'bilevel optimization',
 'optical character recognition': 'object recognition',
 'time series': 'real-time strategy games',
 'car racing': 'autonomous vehicles',
 'dimensionality reduction': 'sparse learning',
 'pose estimation': 'density estimation',
 'knowledge graphs': 'dependency parsing',
 'machine translation': 'speech-to-text translation',
 'text matching': 'semantic parsing',
 'hyperparameter optimization': 'bilevel optimization',
 'information retrieval': 'semantic segmentation',
 'word alignment': 'word sense disambiguation',
 'natural language inference': 'natural language understanding',
 'variational inference': 'bilevel optimization',
 'boundary detection': 'lane detection',
 'point processes': 'latent variable models',
 'sentiment analysis': 'lexical analysis',
 'named entity recognition': 'entity linkin

In [17]:
gensim.models.Word2Vec.load('output/abstract_w2v.bin')

INFO - 19:02:03: loading Word2Vec object from output/abstract_w2v.bin
INFO - 19:02:03: loading wv recursively from output/abstract_w2v.bin.wv.* with mmap=None
INFO - 19:02:03: loading vectors from output/abstract_w2v.bin.wv.vectors.npy with mmap=None
INFO - 19:02:04: setting ignored attribute vectors_norm to None
INFO - 19:02:04: loading vocabulary recursively from output/abstract_w2v.bin.vocabulary.* with mmap=None
INFO - 19:02:04: loading trainables recursively from output/abstract_w2v.bin.trainables.* with mmap=None
INFO - 19:02:04: loading syn1neg from output/abstract_w2v.bin.trainables.syn1neg.npy with mmap=None
INFO - 19:02:04: setting ignored attribute cum_table to None
INFO - 19:02:04: loaded output/abstract_w2v.bin
